## Profiling and Timing Code

In the process of developing coe and creating data processing pipelines, there are often trade-offs you can make between various implementations.  Early in developing your algorithm, it can be counterproductive to worry about such things.  As Donald Knuth famously quipped, "We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil."

But once ou have your code working it can be useful to dig into its efficiency a bit.  Sometimes it's useful to check the execution time of a given command or set of commands.  Other times it's useful to dig into a multiline porcess and determine where the bottleneck lies in some complicated series of operations.  IPython provies access to a wide array of functionality for this kind of timing and profiling of code.  Here we'll discuss the following IPython commands

* `%time`
 * Time the execution of a single statement
* `%timeit`
 * Time repeated execution of a single statement for more accuracy 
* `%prun`
 * Run code with the profiler
* `%lprun`
 * Run code with the line-by-line profiler
* `%memit`
 * Measure the memory use of a single statement
* `%mprun`
 * Run code with the line-by-line memory profiler
 
The last four commands are not bundled with IPython - You'll need to install the `line_profiler` and `memory_profiler` extentions

In [1]:
%timeit sum(range(100))

2.92 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Because this operation is so fast, `%timeit` automatically does a large number of repetitions.  For slower commands, `%timeit` will automatically adjust and perform fewer repetitions

In [2]:
%%timeit
total =  0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

1.06 s ± 18.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Sometimes repeating an operation is not the best option.  For example, if we have a list that we'dlike to sort, we might be misled by a prepeated operation.  Sorting a presorted list is much faster than sorting an unsorted list, so the repetition will skew the result:

In [3]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

2.55 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


For this, the `%time` function may be a better choice.  It also is a good choice for longer-running commands, when short, system-related delays are unlikely to affect the result.  Let's time the sorting of an  unsorted and presorted list

In [4]:
L = [random.random() for i in range(100000)]
print("Sorting an unsorted list:")
%time L.sort()

Sorting an unsorted list:
Wall time: 33.6 ms


In [5]:
print("Sorting an already sorted list: ")
%time L.sort()

Sorting an already sorted list: 
Wall time: 3 ms


Notice how `%timeit` is way quicker, even for a than a presorted list.  This is a result of the fact that `%timeit` does some clever things under the hood to prevent system calls from interfering with the timing.  For example, it prevents cleanup of unused Python objects (known as *garbage collection*) that might otherwise affect the timing results.  

For `%time` as with `%timeit`, using double peprcent sign cell magic syntax allows timing of multiline scripts: 

In [6]:
%%time

total = 0
for i in range(1000):
    for j in range(1000):
        total += i*(-1)**j

Wall time: 1.25 s


For more information on these functions as well as their available options, use the IPython help functionality (i.e., type `%time?`)

### Profiling Full Scripts: `%prun`

A program is mae of many single statements, and sometimes timeing these statements in context is more important than timing them on their own.  Python contains a built-in code profiler (which you can read about in the Python documentation), but IPython offers a much more convenient way to use this profilerm in the form of the magic function `%prun`

By way of example, we'll define a simple function that does some calcuations

In [7]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

Now we can call `%prun` with a function call to see the profiled results:

In [8]:
%prun sum_of_lists(100000)

The result is a table that indicates, in order of total time on each function call, where the execution is spending the most time.  In this case, the bulk of execution time is in the list comprehension inside sum_of_lists.  From here, we could start thinking oabout what changes we might make to improve the performance in the algorithm.

For more information on `%prun` type: `%prun?`

### Line-by-Line Profiling with `%lprun`

Needed to install `line_profiler` first

This is the kind of like running `library()` in r, or like `import package`.  You need to do it in orer to run the magic command

In [9]:
%load_ext line_profiler

Now the `%lprun` command will do a line by line profiling of any function - in this case, we need to tell it explicitly wich functions we are interested in profiling

In [10]:
%lprun -f sum_of_lists sum_of_lists(5000)

The time is reported in microseconds and we can see where the program is spending the most time.  At this point we may be able to use this info to modify aspects of the script and make it perfor better for our desired use case

### Profiling Memory Use: `%memit` and `%mprun`

Another aspect of profiling is the amout of memory an operation uses.  This ca nbe evaluated with another IPython extention, the `memory_profiler`. 

In [11]:
%load_ext memory_profiler

The memory profiler extension contains two useful magic functions: the `%memit` magic (which offers a memory-measuring equivalent of `%timeit`) and the `%mprun` fuction (which offers a memory-measuring equivalet of `%lprun`).

In [12]:
%memit sum_of_lists(1000000)

peak memory: 130.81 MiB, increment: 71.92 MiB


For a line-by-line description of memory use, we can use the `%mprun` magic.  Unfortunately this magic works only for functions defined in separate modules rather than the notebook itself, so we'll start by using the `%%file` magic to create a simple module called `mprun_demo.py` which contains our `sum_of_lists` function, with on addition that will make our memory profiling results more clear:

In [13]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Overwriting mprun_demo.py


We can now import the new version of this function and run the memory line profiler:

This one takes a bit

In [14]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)

Here the Increment column tells us how much each line affects the total memory budget: observe thtat when we create and delete the list L, we are adding about 25 MB of memory usage.  This is on top of the background memory usage from the Python interpreter itself.